**1. Dependencies and Setup**

In [1]:
!pip install catboost tslearn

**2. Importing Libraries**

In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.decomposition import PCA
from scipy import stats
from sklearn.preprocessing import StandardScaler
import matplotlib.dates as mdates
from tslearn.preprocessing import TimeSeriesScalerMeanVariance

from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, classification_report

**3. Data Loading**

In [3]:
df = pd.read_csv("final_data_clipped.csv")

print(df.head())

<ipython-input-3-9f82b15b9ae4>:1: DtypeWarning: Columns (38,51,53,54,84,108,122,139,142,159,160,161) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv("final_data_clipped.csv")


   Unnamed: 0  patient_id    pr_display spo2_display resp_display  \
0           0        7001  71 beats/min          96%          17%   
1           1        7001  71 beats/min          96%          17%   
2           2        7001  71 beats/min          96%          17%   
3           3        7001  71 beats/min          96%          17%   
4           4        7001  71 beats/min          96%          17%   

   pulse_rate_obscount  pulse_rate_avg  pulse_rate_min  pulse_rate_max  \
0                 10.0           70.01           66.63           71.52   
1                 10.0           70.01           66.63           71.52   
2                 10.0           70.01           66.63           71.52   
3                 10.0           70.01           66.63           71.52   
4                 10.0           70.01           66.63           71.52   

   pulse_rate_iqr  ...  QC Deviation from median.1  \
0            2.88  ...                    0.047985   
1            2.88  ...          

**4. Exploratory Data Analysis and Cleaning**

In this section, relevant features are identified and subsets are created based on the type of agents (JNJ or BMS).

In [4]:
complete_set = ['PT_ID','CRS on date (0 No, 1 Yes)','Agent (JNJ/BMS/Caribou)','datetime', 'spo2_avg', 'pulse_rate_avg','respiratory_rate_avg', 'covered_skin_temperature_avg','covered_axil_temperature_avg','Highest Ferritin',
 'Highest CRP','IL8',
 'TNFRSF9',
 'TIE2',
 'MCP-3',
 'CD40-L',
 'IL-1 alpha',
 'CD244',
 'EGF',
 'ANGPT1',
 'IL7',
 'PGF',
 'IL6',
 'ADGRG1',
 'MCP-1',
 'CRTAM',
 'CXCL11',
 'MCP-4',
 'TRAIL',
 'FGF2',
 'CXCL9',
 'CD8A',
 'CAIX',
 'MUC-16',
 'ADA',
 'CD4',
 'NOS3',
 'IL2',
 'Gal-9',
 'VEGFR-2',
 'CD40',
 'IL18',
 'GZMH',
 'KIR3DL1',
 'LAP TGF-beta-1',
 'CXCL1',
 'TNFSF14',
 'IL33',
 'TWEAK',
 'PDGF subunit B',
 'PDCD1',
 'FASLG',
 'CD28',
 'CCL19',
 'MCP-2',
 'CCL4',
 'IL15',
 'Gal-1',
 'PD-L1',
 'CD27',
 'CXCL5',
 'IL5',
 'HGF',
 'GZMA',
 'HO-1',
 'CX3CL1',
 'CXCL10',
 'CD70',
 'IL10',
 'TNFRSF12A',
 'CCL23',
 'CD5',
 'CCL3',
 'MMP7',
 'ARG1',
 'NCR1',
 'DCN',
 'TNFRSF21',
 'TNFRSF4',
 'MIC-A/B',
 'CCL17',
 'ANGPT2',
 'PTN',
 'CXCL12',
 'IFN-gamma',
 'LAMP3',
 'CASP-8',
 'ICOSLG',
 'MMP12',
 'CXCL13',
 'PD-L2',
 'VEGFA',
 'IL4',
 'LAG3',
 'IL12RB1',
 'IL13',
 'CCL20',
 'TNF',
 'KLRD1',
 'GZMB',
 'CD83',
 'IL12',
 'CSF-1',]

Taking different feature sets for JNJ and BMS

In [5]:
columns_test_JNJ = ['Agent (JNJ/BMS/Caribou)', 'CAIX', 'CASP-8', 'CCL23', 'CD40-L', 'CD70',
'CRS on date (0 No, 1 Yes)', 'CXCL10', 'CXCL11', 'CXCL13', 'FASLG',
'FGF2', 'GZMB', 'GZMH', 'Highest CRP', 'Highest Ferritin', 'IFN-gamma',
'IL10', 'IL13', 'IL15', 'IL6', 'IL8', 'MCP-2', 'MMP12', 'PT_ID',
'TIE2', 'TNFRSF9', 'TNFSF14', 'covered_skin_temperature_avg', 'datetime',
'pulse_rate_avg', 'respiratory_rate_avg', 'spo2_avg']
columns_test_BMS = ['FASLG', 'MCP-1', 'CD8A', 'CD70', 'CCL19', 'Highest CRP', 'KLRD1', 'TNFRSF9', 'CXCL12', 'ADGRG1', 'IL2', 'CXCL11', 'GZMH', 'TRAIL', 'IL5', 'TNFSF14', 'HO-1', 'CXCL1', 'CXCL5', 'CD244',
 'PT_ID', 'CRS on date (0 No, 1 Yes)', 'Agent (JNJ/BMS/Caribou)', 'datetime', 'spo2_avg', 'pulse_rate_avg', 'respiratory_rate_avg', 'covered_skin_temperature_avg', 'IL8', 'IL6', 'CXCL10',
 'IFN-gamma', 'CCL23', 'CASP-8', 'CXCL13']
df_subset_JNJ = df[columns_test_JNJ]
df_subset_BMS = df[columns_test_BMS]
df_subset_JNJ.head(5)
df_subset_BMS.head(5)

,FASLG,MCP-1,CD8A,CD70,CCL19,Highest CRP,KLRD1,TNFRSF9,CXCL12,ADGRG1,...,pulse_rate_avg,respiratory_rate_avg,covered_skin_temperature_avg,IL8,IL6,CXCL10,IFN-gamma,CCL23,CASP-8,CXCL13
0,5.806320,11.550530,8.241330,3.730040,10.739640,22.600000,4.535950,5.676640,2.734640,2.15725,...,70.01,24.35,27.650,5.570850,4.754540,9.135090,5.938620,10.893300,4.387440,6.721580
1,5.806320,11.550530,8.241330,3.730040,10.739640,22.600000,4.535950,5.676640,2.734640,2.15725,...,70.01,24.35,27.650,5.570850,4.754540,9.135090,5.938620,10.893300,4.387440,6.721580
2,5.810654,11.572441,8.239465,3.734411,10.745863,22.574747,4.536448,5.677832,2.735678,2.15513,...,70.01,19.86,27.735,5.575702,4.773422,9.158341,6.003697,10.897528,4.403390,6.724164
3,5.814989,11.594352,8.237599,3.738781,10.752086,22.549495,4.536946,5.679023,2.736717,2.15301,...,70.01,15.37,27.820,5.580553,4.792303,9.181591,6.068774,10.901756,4.419340,6.726747
4,5.819323,11.616262,8.235734,3.743152,10.758310,22.524242,4.537444,5.680215,2.737755,2.15089,...,70.01,11.77,27.780,5.585405,4.811185,9.204842,6.133852,10.905983,4.435289,6.729331


In [6]:
df_JNJ = df_subset_JNJ[(df_subset_JNJ['Agent (JNJ/BMS/Caribou)']=='JNJ') | (df_subset_JNJ['Agent (JNJ/BMS/Caribou)']=='JNJ OOS')]
df_BMS = df_subset_BMS[(df_subset_BMS['Agent (JNJ/BMS/Caribou)']=='BMS')]

**5. Data Individualization and Baseline Adjustment**

Columns for BMS are chosen here.

In [7]:
# Individualize the data
# Create a baseline dataframe
data = df_BMS.copy()

baseline = data.groupby('PT_ID').first().reset_index()

# Subtract the baseline values for numeric columns
columns_to_individualize = [col for col in columns_test_BMS if col not in ['PT_ID', 'CRS on date (0 No, 1 Yes)', 'Agent (JNJ/BMS/Caribou)','datetime']]

for col in columns_to_individualize:
    data[col] = pd.to_numeric(data[col], errors='coerce')
    data[col] = data[col] - data.groupby('PT_ID')[col].transform('first')

data.head()

,FASLG,MCP-1,CD8A,CD70,CCL19,Highest CRP,KLRD1,TNFRSF9,CXCL12,ADGRG1,...,pulse_rate_avg,respiratory_rate_avg,covered_skin_temperature_avg,IL8,IL6,CXCL10,IFN-gamma,CCL23,CASP-8,CXCL13
0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,...,0.0,0.00,0.000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
1,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,...,0.0,0.00,0.000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
2,0.004334,0.021911,-0.001865,0.004371,0.006223,-0.025253,0.000498,0.001192,0.001038,-0.00212,...,0.0,-4.49,0.085,0.004852,0.018882,0.023251,0.065077,0.004228,0.015950,0.002584
3,0.008669,0.043822,-0.003731,0.008741,0.012446,-0.050505,0.000996,0.002383,0.002077,-0.00424,...,0.0,-8.98,0.170,0.009703,0.037763,0.046501,0.130154,0.008456,0.031900,0.005167
4,0.013003,0.065732,-0.005596,0.013112,0.018670,-0.075758,0.001494,0.003575,0.003115,-0.00636,...,0.0,-12.58,0.130,0.014555,0.056645,0.069752,0.195232,0.012683,0.047849,0.007751


**6. Feature Engineering: Rolling and Lagged Features**

Additional features are generated to capture short-term trends and variability in the measurements. Past values and rolling statistics over a 8-hour window are computed to provide temporal context for the prediction model.

In [8]:
# Define the window size for rolling statistics (e.g., past 6 hours)
window_size = 8
lag_size = 8
time_interval = 30

# Sort data by patient ID and datetime
data = data.sort_values(by=['PT_ID', 'datetime'])

# Function to create lagged features and rolling statistics
def add_past_features(data, columns,lag_size):
    for col in columns:
        # Add lagged values
        for lag_base in range(1, lag_size + 1):
            lag = lag_base * time_interval
            data[f'{col}_lag_{lag}'] = data.groupby('PT_ID')[col].shift(lag)

        rolling_size = lag_size * time_interval
        # Add rolling statistics
        data[f'{col}_rolling_mean_{rolling_size}'] = data.groupby('PT_ID')[col].rolling(rolling_size, min_periods=1).mean().reset_index(level=0, drop=True)
        data[f'{col}_rolling_std_{rolling_size}'] = data.groupby('PT_ID')[col].rolling(rolling_size, min_periods=1).std().reset_index(level=0, drop=True)
        data[f'{col}_rolling_min_{rolling_size}'] = data.groupby('PT_ID')[col].rolling(rolling_size, min_periods=1).min().reset_index(level=0, drop=True)
        data[f'{col}_rolling_max_{rolling_size}'] = data.groupby('PT_ID')[col].rolling(rolling_size, min_periods=1).max().reset_index(level=0, drop=True)


    return data

# Add past features for selected columns
columns_to_process = [col for col in columns_test_BMS if col not in ['PT_ID','CRS on date (0 No, 1 Yes)','Agent (JNJ/BMS/Caribou)','datetime']]
data = add_past_features(data, columns_to_process, lag_size)

<ipython-input-8-537fa873deb7>:15: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  data[f'{col}_lag_{lag}'] = data.groupby('PT_ID')[col].shift(lag)
<ipython-input-8-537fa873deb7>:15: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  data[f'{col}_lag_{lag}'] = data.groupby('PT_ID')[col].shift(lag)
<ipython-input-8-537fa873deb7>:15: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1)

**7. Creating column: CRS in 8 Hours**

In this section, a binary target variable CRS_in_8_hours is created. It indicates whether a patient will experience CRS within the next 8 hours from any given measurement time.

In [9]:
from datetime import timedelta

def assign_crs_in_8_hours(data):
    """
    Assign CRS_in_8_hours for each row based on whether `datetime + 8 hours` falls within a CRS occurrence time frame.

    Parameters:
        data (DataFrame): Input DataFrame with 'PT_ID', 'datetime', and 'CRS on date (0 No, 1 Yes)' columns.

    Returns:
        DataFrame: Updated DataFrame with a new column 'CRS_in_8_hours'.
    """
    # Ensure 'datetime' is a datetime object
    data['datetime'] = pd.to_datetime(data['datetime'])
    data = data.sort_values(by=['PT_ID', 'datetime'])

    # Initialize a new column
    data['CRS_in_8_hours'] = 0

    # Process each patient group separately
    for pt_id, group in data.groupby('PT_ID'):
        # Sort by datetime for the current patient
        group = group.sort_values('datetime')

        # Identify CRS occurrence start and end timeframes
        crs_start = group.index[(group['CRS on date (0 No, 1 Yes)'].shift(1) == 0) &
                                (group['CRS on date (0 No, 1 Yes)'] == 1)].tolist()
        crs_end = group.index[(group['CRS on date (0 No, 1 Yes)'].shift(1) == 1) &
                              (group['CRS on date (0 No, 1 Yes)'] == 0)].tolist()

        # If a CRS event starts but does not end, assume it continues until the last datetime
        if len(crs_start) > len(crs_end):
            crs_end.append(group.index[-1])

        # Assign CRS_in_8_hours for each row
        for start_idx, end_idx in zip(crs_start, crs_end):
            crs_start_time = group.loc[start_idx, 'datetime']
            crs_end_time = group.loc[end_idx, 'datetime']

            # Any datetime + 8 hours within the CRS occurrence timeframe is set to 1
            within_crs_timeframe = (group['datetime'] + timedelta(hours=6) >= crs_start_time) & \
                                   (group['datetime'] + timedelta(hours=6) <= crs_end_time)
            data.loc[group[within_crs_timeframe].index, 'CRS_in_8_hours'] = 1

    return data

# Apply the function to your dataset
data = assign_crs_in_8_hours(data)



In [10]:
data[data['CRS_in_8_hours']==1]

,FASLG,MCP-1,CD8A,CD70,CCL19,Highest CRP,KLRD1,TNFRSF9,CXCL12,ADGRG1,...,CXCL13_lag_120,CXCL13_lag_150,CXCL13_lag_180,CXCL13_lag_210,CXCL13_lag_240,CXCL13_rolling_mean_240,CXCL13_rolling_std_240,CXCL13_rolling_min_240,CXCL13_rolling_max_240,CRS_in_8_hours
1,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,NaN,NaN,NaN,NaN,NaN,0.000000,0.000000,0.000000,0.000000,1
2,0.004334,0.021911,-0.001865,0.004371,0.006223,-0.025253,0.000498,0.001192,0.001038,-0.002120,...,NaN,NaN,NaN,NaN,NaN,0.000861,0.001492,0.000000,0.002584,1
3,0.008669,0.043822,-0.003731,0.008741,0.012446,-0.050505,0.000996,0.002383,0.002077,-0.004240,...,NaN,NaN,NaN,NaN,NaN,0.001938,0.002474,0.000000,0.005167,1
4,0.013003,0.065732,-0.005596,0.013112,0.018670,-0.075758,0.001494,0.003575,0.003115,-0.006360,...,NaN,NaN,NaN,NaN,NaN,0.003100,0.003369,0.000000,0.007751,1
5,0.017337,0.087643,-0.007461,0.017482,0.024893,-0.101010,0.001992,0.004766,0.004153,-0.008480,...,NaN,NaN,NaN,NaN,NaN,0.004306,0.004219,0.000000,0.010334,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
320609,0.221439,-1.509934,0.740310,-0.222933,1.218298,42.411220,-0.109863,0.124117,0.155200,-0.547634,...,1.337700,1.314459,1.291218,1.267977,1.244737,1.338087,0.053784,1.245511,1.430662,1
320610,0.221551,-1.511560,0.740584,-0.223324,1.217921,42.411463,-0.109800,0.124070,0.154933,-0.547844,...,1.338474,1.315233,1.291993,1.268752,1.245511,1.338862,0.053784,1.246286,1.431437,1
320611,0.221664,-1.513187,0.740858,-0.223715,1.217544,42.411707,-0.109737,0.124024,0.154666,-0.548054,...,1.339249,1.316008,1.292767,1.269527,1.246286,1.339636,0.053784,1.247061,1.432212,1
320612,0.221777,-1.514813,0.741132,-0.224105,1.217167,42.411951,-0.109673,0.123977,0.154399,-0.548263,...,1.340024,1.316783,1.293542,1.270301,1.247061,1.340411,0.053784,1.247835,1.432987,1


**8. Model Training and Evaluation**

In this section, the dataset is split by patients into training and test sets using K-fold cross-validation. Several models (LightGBM, CatBoost, XGBoost) are trained to predict CRS_in_8_hours. Random oversampling is used to handle class imbalance. The performance is evaluated using accuracy, AUC-ROC, and classification reports.

**a. LightGBM Model with Oversampling**


A LightGBM classifier is trained and evaluated with cross-validation.

In [11]:
import pandas as pd
import numpy as np
from sklearn.model_selection import KFold
from sklearn.metrics import classification_report, roc_auc_score, accuracy_score, confusion_matrix
from imblearn.over_sampling import RandomOverSampler
from lightgbm import LGBMClassifier

if 'datetime' in data.columns:
    data = data.drop(columns=['datetime'])
if 'cutoff_date' in data.columns:
    data = data.drop(columns=['cutoff_date'])

# Define columns to drop (identifiers, target)
drop_cols = ['PT_ID', 'CRS_in_8_hours', 'CRS on date (0 No, 1 Yes)', 'Agent (JNJ/BMS/Caribou)']
feature_cols = [col for col in data.columns if col not in drop_cols]

unique_patients = data['PT_ID'].unique()
kf = KFold(n_splits=5, shuffle=False)

cv_accuracies = []
cv_auc_scores = []
aggregate_conf_matrix = np.array([[0, 0],
                                  [0, 0]])

for fold, (train_idx, test_idx) in enumerate(kf.split(unique_patients)):
    print(f"\nFold {fold + 1}/5")

    train_patients = unique_patients[train_idx]
    test_patients = unique_patients[test_idx]

    train_data = data[data['PT_ID'].isin(train_patients)]
    test_data = data[data['PT_ID'].isin(test_patients)]

    X_train = train_data[feature_cols]
    y_train = train_data['CRS_in_8_hours']

    X_test = test_data[feature_cols]
    y_test = test_data['CRS_in_8_hours']

    # Handle class imbalance with Random Oversampling
    oversampler = RandomOverSampler(random_state=42)
    X_train_resampled, y_train_resampled = oversampler.fit_resample(X_train, y_train)

    print("Class distribution after oversampling:")
    print(pd.Series(y_train_resampled).value_counts())


    lgbm_model = LGBMClassifier(
        objective='binary',
        max_depth=6,
        learning_rate=0.1,
        n_estimators=100,
        random_state=42
    )

    # Train the model
    lgbm_model.fit(X_train_resampled, y_train_resampled)

    # Predict on test set
    y_pred = lgbm_model.predict(X_test)
    y_prob = lgbm_model.predict_proba(X_test)[:, 1]

    # Calculate metrics
    accuracy = accuracy_score(y_test, y_pred)
    auc_score = roc_auc_score(y_test, y_prob)
    conf_matrix = confusion_matrix(y_test, y_pred)

    print("Accuracy:", accuracy)
    print("AUC-ROC Score:", auc_score)
    print("Classification Report:\n", classification_report(y_test, y_pred))

    cv_accuracies.append(accuracy)
    cv_auc_scores.append(auc_score)
    aggregate_conf_matrix += conf_matrix

# Aggregate results
print("\nCross-Validation Results:")
print(f"Average Accuracy: {np.mean(cv_accuracies)}")
print(f"Average AUC-ROC: {np.mean(cv_auc_scores)}")
print("Aggregated Confusion Matrix:\n", aggregate_conf_matrix)

tn, fp, fn, tp = aggregate_conf_matrix.ravel()
class0_accuracy = tn / (tn + fp) if (tn + fp) > 0 else 0
class1_accuracy = tp / (fn + tp) if (fn + tp) > 0 else 0

print(f"Accuracy for class 0: {class0_accuracy}")
print(f"Accuracy for class 1: {class1_accuracy}")


/usr/local/lib/python3.10/dist-packages/dask/dataframe/__init__.py:42: FutureWarning: 
Dask dataframe query planning is disabled because dask-expr is not installed.

You can install it with `pip install dask[dataframe]` or `conda install dask`.
This will raise in a future version.

  warnings.warn(msg, FutureWarning)



Fold 1/5
Class distribution after oversampling:
CRS_in_8_hours
0    69255
1    69255
Name: count, dtype: int64
[LightGBM] [Warning] Found whitespace in feature_names, replace with underlines
[LightGBM] [Info] Number of positive: 69255, number of negative: 69255
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.060125 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 99131
[LightGBM] [Info] Number of data points in the train set: 138510, number of used features: 403
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.500000 -> initscore=0.000000
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
Accuracy: 0.5824300389656394
AUC-ROC Score: 0.5135230788391146
Classification Report:
               precision    recall  f1-score   support

           0       0.86      0.62      0.72     24576
           1       0.11      0.

**b. CatBoost Model**


CatBoost classifier is also tested. Class weights are used to handle class imbalance, and a similar cross-validation approach is followed.

In [12]:
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import classification_report, roc_auc_score, accuracy_score, recall_score
from catboost import CatBoostClassifier
import pandas as pd
import numpy as np

catboost_model = CatBoostClassifier(
    loss_function='Logloss',
    max_depth=6,
    learning_rate=0.1,
    iterations=100,
    random_seed=42,
    verbose=False,
    class_weights=[1, 2.0]
)


# Stratified K-Fold Cross-Validation without shuffling
kf = StratifiedKFold(n_splits=5, shuffle=False)

fold_metrics = []
fold = 1

for train_idx, test_idx in kf.split(X_train, y_train):
    print(f"Fold {fold}/5")

    # Split data into train and test sets for this fold
    X_fold_train, X_fold_test = X_train.iloc[train_idx], X_train.iloc[test_idx]
    y_fold_train, y_fold_test = y_train.iloc[train_idx], y_train.iloc[test_idx]

    # Class distributions
    print(f"Training Class Distribution:\n{y_fold_train.value_counts()}")
    print(f"Validation Class Distribution:\n{y_fold_test.value_counts()}")

    # Train the CatBoost model directly with class weights
    catboost_model.fit(X_fold_train, y_fold_train)

    # Predict probabilities
    y_prob = catboost_model.predict_proba(X_fold_test)[:, 1]

    # Threshold search for best balanced accuracy
    thresholds = np.linspace(0, 1, 101)
    best_bal_acc = 0.0
    best_threshold = 0.5

    for th in thresholds:
        y_pred_th = (y_prob >= th).astype(int)
        rec_class0 = recall_score(y_fold_test, y_pred_th, pos_label=0)
        rec_class1 = recall_score(y_fold_test, y_pred_th, pos_label=1)
        bal_acc = 0.5 * (rec_class0 + rec_class1)
        if bal_acc > best_bal_acc:
            best_bal_acc = bal_acc
            best_threshold = th

    # Use best threshold
    y_pred = (y_prob >= best_threshold).astype(int)

    acc = accuracy_score(y_fold_test, y_pred)
    auc = roc_auc_score(y_fold_test, y_prob)
    report = classification_report(y_fold_test, y_pred, digits=2, output_dict=True)

    # Extract metrics
    f1_class0 = report["0"]["f1-score"]
    f1_class1 = report["1"]["f1-score"]
    recall_class0 = report["0"]["recall"]
    recall_class1 = report["1"]["recall"]

    fold_metrics.append({
        "Fold": fold,
        "Best Threshold": best_threshold,
        "Accuracy": acc,
        "AUC-ROC": auc,
        "F1-Score (Class 0)": f1_class0,
        "F1-Score (Class 1)": f1_class1,
        "Recall (Class 0)": recall_class0,
        "Recall (Class 1)": recall_class1,
        "Balanced Accuracy": best_bal_acc
    })

    # Print fold results
    print(f"Best Threshold: {best_threshold:.3f}")
    print(f"Accuracy: {acc:.4f}")
    print(f"AUC-ROC: {auc:.4f}")
    print(f"Balanced Accuracy: {best_bal_acc:.4f}")
    print("Classification Report:")
    print(classification_report(y_fold_test, y_pred, digits=2))
    print("-" * 50)

    fold += 1

fold_metrics_df = pd.DataFrame(fold_metrics)
print("\nCross-Validation Results:")
print(fold_metrics_df)

# Compute averages of metrics
avg_threshold = fold_metrics_df["Best Threshold"].mean()
avg_f1_class1 = fold_metrics_df["F1-Score (Class 1)"].mean()
avg_accuracy = fold_metrics_df["Accuracy"].mean()
avg_auc = fold_metrics_df["AUC-ROC"].mean()
avg_recall_class0 = fold_metrics_df["Recall (Class 0)"].mean()
avg_recall_class1 = fold_metrics_df["Recall (Class 1)"].mean()
avg_bal_acc = fold_metrics_df["Balanced Accuracy"].mean()

print("\nCross-Validation Results:")
print(f"Average Accuracy: {avg_accuracy}")
print(f"Average AUC-ROC: {avg_auc}")
print(f"Average Accuracy (Class 0): {avg_recall_class0}")
print(f"Average Accuracy (Class 1): {avg_recall_class1}")


Fold 1/5
Training Class Distribution:
CRS_in_8_hours
0    65580
1    14556
Name: count, dtype: int64
Validation Class Distribution:
CRS_in_8_hours
0    16395
1     3639
Name: count, dtype: int64
Best Threshold: 0.170
Accuracy: 0.5967
AUC-ROC: 0.6888
Balanced Accuracy: 0.7239
Classification Report:
              precision    recall  f1-score   support

           0       0.97      0.52      0.68     16395
           1       0.30      0.92      0.45      3639

    accuracy                           0.60     20034
   macro avg       0.63      0.72      0.57     20034
weighted avg       0.85      0.60      0.64     20034

--------------------------------------------------
Fold 2/5
Training Class Distribution:
CRS_in_8_hours
0    65580
1    14556
Name: count, dtype: int64
Validation Class Distribution:
CRS_in_8_hours
0    16395
1     3639
Name: count, dtype: int64
Best Threshold: 0.030
Accuracy: 0.9157
AUC-ROC: 0.9416
Balanced Accuracy: 0.9472
Classification Report:
              precision 

**c. XGBoost Model Training**


In this section, an XGBoost model is trained and evaluated using the same methodology. Random oversampling is applied and performance metrics are calculated to compare against other models.

In [13]:
import pandas as pd
import numpy as np
from sklearn.model_selection import KFold
from sklearn.metrics import classification_report, roc_auc_score, accuracy_score, confusion_matrix
from imblearn.over_sampling import RandomOverSampler
import xgboost as xgb

if 'cutoff_date' in data.columns:
    data = data.drop(columns=['cutoff_date'])

if 'datetime' in data.columns:
    data = data.drop(columns=['datetime'])

drop_cols = ['PT_ID', 'CRS on date (0 No, 1 Yes)', 'Agent (JNJ/BMS/Caribou)', 'CRS_in_8_hours']
feature_cols = [col for col in data.columns if col not in drop_cols]

unique_patients = data['PT_ID'].unique()
kf = KFold(n_splits=5, shuffle=False)

cv_accuracies = []
cv_auc_scores = []
cv_classification_reports = []
aggregate_conf_matrix = np.array([[0, 0],
                                  [0, 0]])

for fold, (train_idx, test_idx) in enumerate(kf.split(unique_patients)):
    print(f"\nFold {fold + 1}/5")

    train_patients = unique_patients[train_idx]
    test_patients = unique_patients[test_idx]

    train_data = data[data['PT_ID'].isin(train_patients)]
    test_data = data[data['PT_ID'].isin(test_patients)]

    X_train = train_data[feature_cols]
    y_train = train_data['CRS_in_8_hours']

    X_test = test_data[feature_cols]
    y_test = test_data['CRS_in_8_hours']


    # Random Oversampling
    oversampler = RandomOverSampler(random_state=42)
    X_train_resampled, y_train_resampled = oversampler.fit_resample(X_train, y_train)

    dtrain = xgb.DMatrix(X_train_resampled, label=y_train_resampled)
    dtest = xgb.DMatrix(X_test, label=y_test)

    params = {
        'objective': 'binary:logistic',
        'eval_metric': 'logloss',
        'max_depth': 6,
        'learning_rate': 0.1
    }

    model = xgb.train(params, dtrain, num_boost_round=100)

    y_prob = model.predict(dtest)
    threshold = 0.01
    y_pred = (y_prob > threshold).astype(int)

    accuracy = accuracy_score(y_test, y_pred)
    auc_score = roc_auc_score(y_test, y_prob)
    conf_matrix = confusion_matrix(y_test, y_pred)
    class_report = classification_report(y_test, y_pred, output_dict=True)

    print("Accuracy:", accuracy)
    print("AUC-ROC Score:", auc_score)
    print("Classification Report:\n", classification_report(y_test, y_pred))

    cv_accuracies.append(accuracy)
    cv_auc_scores.append(auc_score)
    cv_classification_reports.append(class_report)
    aggregate_conf_matrix += conf_matrix

print("\nCross-Validation Results:")
print(f"Average Accuracy: {np.mean(cv_accuracies)}")
print(f"Average AUC-ROC: {np.mean(cv_auc_scores)}")
print("Aggregated Confusion Matrix:\n", aggregate_conf_matrix)

tn, fp, fn, tp = aggregate_conf_matrix.ravel()
class0_accuracy = tn / (tn + fp) if (tn + fp) > 0 else 0
class1_accuracy = tp / (fn + tp) if (fn + tp) > 0 else 0

print(f"Accuracy for class 0: {class0_accuracy}")
print(f"Accuracy for class 1: {class1_accuracy}")


Fold 1/5
Accuracy: 0.5255047821466525
AUC-ROC Score: 0.5606415568857901
Classification Report:
               precision    recall  f1-score   support

           0       0.85      0.55      0.67     24576
           1       0.10      0.34      0.16      3654

    accuracy                           0.53     28230
   macro avg       0.48      0.45      0.41     28230
weighted avg       0.75      0.53      0.60     28230


Fold 2/5
Accuracy: 0.8002987582858744
AUC-ROC Score: 0.7019466328063364
Classification Report:
               precision    recall  f1-score   support

           0       0.79      0.97      0.87     15187
           1       0.84      0.39      0.53      6235

    accuracy                           0.80     21422
   macro avg       0.82      0.68      0.70     21422
weighted avg       0.81      0.80      0.77     21422


Fold 3/5
Accuracy: 0.7211121295447601
AUC-ROC Score: 0.7392162357809995
Classification Report:
               precision    recall  f1-score   support



Results Evaluation

Among the three models, **CatBoost** provides the best overall balance of performance for BMS 8-hour CRS prediction. While CatBoost’s raw accuracy is slightly lower than LightGBM’s, its AUC-ROC is higher, indicating stronger discriminative capability. More importantly, it achieves a far better balance between classes, with ~0.73 accuracy for class 0 and ~0.80 for class 1. This improvement shows CatBoost is much more effective at catching the minority class cases without completely sacrificing majority class performance. The model is therefore more clinically valuable, as it reduces the number of missed CRS-onset cases.